Group Members:
1. Vikas Kalyanapuram - IMT2021040
2. Barath S Narayan - IMT2021524

GlobalSetup() - Will try to choose primes such that the n > 2MP^2 condition is satisfied and will set k as the number of primes chosen.

GlobalSetup2() - Will choose k at random and will choose the first k primes irrespective of whether the condition is satisfied or not. 

In [ ]:
!pip3 install gmpy2
from gmpy2 import mpz,mpq,next_prime
import random
from math import floor

In [ ]:

def binary_egcd(a,b):
    
    if(a<b):
        a,b = b,a
    
    r = mpz(a)
    r_d = mpz(b)
    e = mpz(0)
    
    while (((r % 2) == 0) and ((r_d % 2) == 0)):
        r = r >> 1
        r_d = r_d >> 1
        e += 1
    
    a_t = r
    b_t = r_d
    s,t,s_d,t_d = mpz(1),mpz(0),mpz(0),mpz(1)
    
    while (r_d != 0):
        
        while((r % 2) == 0):
            r = r >> 1
            if ((s % 2) == 0 and ((t % 2) == 0)):
                s = s >> 1
                t = t >> 1
            else:
                s = (s + b_t) >> 1
                t = (t - a_t) >> 1
        
        while((r_d % 2) == 0):
            r_d = r_d >> 1
            if (((s_d % 2) == 0) and ((t_d % 2) == 0)):
                s_d = s_d >> 1
                t_d = t_d >> 1
            else:
                s_d = (s_d + b_t) >> 1
                t_d = (t_d - a_t) >> 1

        if(r_d < r):
            r, s, t, r_d, s_d, t_d = r_d, s_d, t_d, r, s, t
        
        r_d = r_d - r
        s_d = s_d - s
        t_d = t_d - t
    
    d = ((1 << e) * r)
    return (d,s,t)

In [ ]:
def egcd(a,b,r_star):
    
    if(a<b):
        a,b = b,a

    r = mpz(a)
    r_d = mpz(b)
    s, s_d, t, t_d = mpz(1), mpz(0), mpz(0), mpz(1)
    while(r_d != 0):
        if(r <= r_star):
            return r,s,t
        q = mpz(r // r_d)
        r_dd = mpz(r % r_d)
        r, s, t, r_d, s_d, t_d = r_d, s_d, t_d, r_dd, mpz(s - (s_d * q)), mpz(t - (t_d * q))
    return (-1,-1,-1)

In [ ]:
def CRT(a,n):
    
    x, N = mpz(0), mpz(1)
    
    for i in n:
        N *= i
    
    for i in range(len(n)):
        
        N_k = mpz(N // n[i])
        (d, x_k, t) = binary_egcd(N_k,n[i])
        
        if(d != mpz(1)):
            x = mpz(-1)
            break
        
        x += mpz( mpz(a[i] * N_k * x_k) % mpz(N) )
    
    return mpz(x) % mpz(N)

In [ ]:
def GlobalSetup(mu,M):
    l = [] #This is the list of primes
    ni = mpz(2)
    product_of_primes = mpz(1)
    while(ni < M and len(l) < 500):
        product_of_primes *= ni
        l.append(ni)
        L = mpz(floor(mu*mpz(len(l))))
        P = mpz(1)
        for j in range(L):
            P *= l[-j - 1]
        if(product_of_primes > 2 * M * P * P):
            break
        ni = next_prime(ni)
    return l

In [ ]:
def GlobalSetup2(k):
    l = []
    ni = mpz(2)
    for i in range(k):
        l.append(ni)
        ni = next_prime(ni)
    return l

In [ ]:
def Transmit(k,n,a,myu):
    b = []
    l = mpz(random.randint(0,mpz(myu*k))) # mpz(myu,k) will turn it into an integer
    I = [] # I is the set of bad indices
    for i in range(0,l):
        x = mpz(random.randint(1,k))
        while x in I: #To generate a unique 'bad' index
            x = mpz(random.randint(1,k))
        I.append(x)
    for i in range(1,k+1):
        if i in I:
            x = mpz(random.randint(0,n[i-1]-1))
            while(x == a[i-1]): # To ensure b[i] != a[i]
                x = mpz(random.randint(0,n[i-1]-1))
            b.append(x)
        else:
            b.append(a[i-1])
    
    return b

In [ ]:
def ReedSolomonSend(a,l,mu):
    msglist = [] # List of ai's
    am = mpz(a)
    for i in range(len(l)):
        msglist.append(mpz(am%l[i]))
    return Transmit(len(l),l,msglist,mu) 

In [ ]:
def ReedSolomonReceive(msglist, n, M, mu):
    P = mpz(1)
    for i in range(floor(mu*len(n))):
        P *= n[-i-1]
    r_star = M * P 
    b = CRT(msglist,n)
    product_of_n = mpz(1)
    for i in n:
        product_of_n *= i    
    r,s,t = egcd(product_of_n,b,r_star)
    if(r == -1 or t == 0 or r % t != 0):
        return "ERROR"
    else:
        return mpz(r//t)

TestCases we tested our code with:

In [ ]:
mu = 0.2
M = mpz(1<<1024)
l = GlobalSetup(mu,M) # l is the list of primes

In [ ]:
a = mpz(72234)
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))

a = mpz(2136834135)
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))

a = mpz(154623461948719238719238774322267944)
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))

a = mpz(1823719472184761)
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))

In [ ]:
mu = 0.2
M = mpz(1<<1024)
l = GlobalSetup2(200) # l is the list of primes

In [ ]:
a = mpz(15)
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))

a = mpz(2136834135)
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))

a = mpz(154623461948719238719238774322267944)
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))

a = mpz(1823719472184761)
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))

Taking input: 

In [ ]:
bits = int(input("Enter number of bits in M:"))
M = mpz(1 << bits)
myu = float(input("Enter myu (<1): "))
l = GlobalSetup(myu,M)

In [ ]:
a = mpz(int(input("Enter Message: ")))
b = ReedSolomonSend(a,l,mu)
print(ReedSolomonReceive(b,l,M,mu))